In [1]:
import torch as th
from torch.autograd import Variable
from torch.nn.modules.loss import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader, TensorDataset
import my
import resnet

In [2]:
train_x, train_y, test_x, test_y = tuple(map(th.from_numpy, my.load_cifar100(rbg=True)))
train_x, test_x = train_x.float(), test_x.float()
N_CLASSES = int(train_y.max() - train_y.min() + 1)

train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=4096)

In [3]:
c = resnet.ResNet(18, N_CLASSES)
cuda = True # always GPU 0
if cuda:
    c.cuda()
optim = SGD(c.parameters(), lr=1e-1, momentum=0.9, weight_decay=1e-4)

In [4]:
EPOCHS = 50
for i in range(EPOCHS):
    for j, (x, y) in enumerate(train_loader):
        if cuda:
            x, y = x.cuda(), y.cuda()
        x, y = Variable(x), Variable(y)
        ce = CrossEntropyLoss()(c(x), y)
        optim.zero_grad()
        ce.backward()
        optim.step()
    test_acc = my.parallel_global_stats(c, test_loader, my.accuracy, range(4))
    print('[epoch %d]cross-entropy loss: %f, accuracy: %f' % ((i + 1), float(ce), float(test_acc)))

[epoch 1]cross-entropy loss: 0.220086, accuracy: 0.907100
[epoch 2]cross-entropy loss: 0.184958, accuracy: 0.915300
[epoch 3]cross-entropy loss: 0.180198, accuracy: 0.916300
[epoch 4]cross-entropy loss: 0.124158, accuracy: 0.930900
[epoch 5]cross-entropy loss: 0.075682, accuracy: 0.935000
[epoch 6]cross-entropy loss: 0.076197, accuracy: 0.940300
[epoch 7]cross-entropy loss: 0.120525, accuracy: 0.948700
[epoch 8]cross-entropy loss: 0.051341, accuracy: 0.945400
[epoch 9]cross-entropy loss: 0.132847, accuracy: 0.954100
[epoch 10]cross-entropy loss: 0.040626, accuracy: 0.955000
[epoch 11]cross-entropy loss: 0.113100, accuracy: 0.954100
[epoch 12]cross-entropy loss: 0.055043, accuracy: 0.953800
[epoch 13]cross-entropy loss: 0.047538, accuracy: 0.950300
[epoch 14]cross-entropy loss: 0.076877, accuracy: 0.955800
[epoch 15]cross-entropy loss: 0.045346, accuracy: 0.951100
[epoch 16]cross-entropy loss: 0.064889, accuracy: 0.957500
[epoch 17]cross-entropy loss: 0.044144, accuracy: 0.953900
[epoch

In [5]:
# for pg in optim.param_groups:
#     pg['lr'] *= 0.1

In [6]:
# th.save(c.state_dict(), 'cifar10-resnet18')
# th.save(c.state_dict(), 'cifar100-1-9-resnet18')